In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

In [27]:
import_source="https://raw.githubusercontent.com/SimbarasheJehol/MachineLearningProject/master/dataProccesed2.csv"
df = pd.read_csv(import_source)
print("data is loaded")

data is loaded


In [29]:
df=df.drop('Unnamed: 0',axis=1)

In [30]:
df=df.drop( "fnlwgt", axis=1)\
                .drop( "capital_gain",axis=1)\
                .drop("capital_loss",axis=1)\
                .drop("education_num_rand",axis=1)\
                .drop( "fnlwgt-binned", axis=1)\
                .drop( "capital_gain-binned",axis=1)\
                .drop("capital_loss-binned",axis=1)\
                .drop("education_num-binned",axis=1)
               

In [52]:
##target_variable=df["income_class"]
df.head()

,age,workclass,education,education_num,martial_status,occupation,RelationShip,race,sex,hours_per_week,native_country,age-binned,hours_per_week-binned,income_class
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,mid_workclass,hours4,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,Advanced_workclass,hours2,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,mid_workclass,hours4,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,Advanced_workclass,hours4,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,junior work_class,hours4,<=50K


In [90]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [91]:
#random.seed(0)// this will always return the same test_df dataframe
train_df, test_df = train_test_split(df, test_size=0.1)

In [92]:
len(test_df)

3256

In [93]:
data = train_df.values ##speeding up the code
data[:5]

array([[39, ' State-gov', ' Bachelors', 13, ' Never-married',
        ' Adm-clerical', ' Not-in-family', ' White', ' Male', 40,
        ' United-States', 'mid_workclass', 'hours4', ' <=50K'],
       [50, ' Self-emp-not-inc', ' Bachelors', 13, ' Married-civ-spouse',
        ' Exec-managerial', ' Husband', ' White', ' Male', 13,
        ' United-States', 'Advanced_workclass', 'hours2', ' <=50K'],
       [38, ' Private', ' HS-grad', 9, ' Divorced', ' Handlers-cleaners',
        ' Not-in-family', ' White', ' Male', 40, ' United-States',
        'mid_workclass', 'hours4', ' <=50K'],
       [53, ' Private', ' 11th', 7, ' Married-civ-spouse',
        ' Handlers-cleaners', ' Husband', ' Black', ' Male', 40,
        ' United-States', 'Advanced_workclass', 'hours4', ' <=50K'],
       [28, ' Private', ' Bachelors', 13, ' Married-civ-spouse',
        ' Prof-specialty', ' Wife', ' Black', ' Female', 40, ' Cuba',
        'junior work_class', 'hours4', ' <=50K']], dtype=object)

In [94]:
def check_purity(data):
    
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [95]:
check_purity(train_df.values)

False

In [96]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [97]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)
        
        # feature is categorical
        # (there need to be at least 2 unique values, otherwise in the
        # split_data function data_below would contain all data points
        # and data_above would be empty)
        elif len(unique_values) > 1:
            potential_splits[column_index] = unique_values
    
    return potential_splits

In [98]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [99]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [100]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [101]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [102]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 20
    for feature in df.columns:
        if feature != "income_class":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [103]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [110]:
tree = decision_tree_algorithm(train_df, max_depth=3)
pprint(tree)

{'martial_status =  Married-civ-spouse': [{'occupation =  Prof-specialty': [{'age <= 32.5': [' '
                                                                                             '<=50K',
                                                                                             ' '
                                                                                             '>50K']},
                                                                            {'occupation =  Exec-managerial': [' '
                                                                                                               '>50K',
                                                                                                               ' '
                                                                                                               '<=50K']}]},
                                          ' <=50K']}


In [111]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [112]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(classify_example, args=(tree,), axis=1)
    df["classification_correct"] = df["classification"] == df["income_class"]
    
    accuracy = df["classification_correct"].mean()
    
    return accuracy

In [113]:
train_df, test_df = train_test_split(df, test_size=0.2)
tree = decision_tree_algorithm(train_df)
accuracy = calculate_accuracy(test_df, tree)

ValueError: ('too many values to unpack (expected 3)', 'occurred at index 18579')

In [107]:
data[21802,:]

array([40, ' Self-emp-not-inc', ' Prof-school', 15, ' Married-civ-spouse',
       ' Prof-specialty', ' Wife', ' White', ' Female', 40,
       ' United-States', 'mid_workclass', 'hours4', ' >50K'], dtype=object)

In [109]:
data[21801,:]

array([45, ' Self-emp-not-inc', ' HS-grad', 9, ' Married-civ-spouse',
       ' Transport-moving', ' Husband', ' White', ' Male', 50,
       ' United-States', 'senior_workclass', 'hours5', ' <=50K'],
      dtype=object)